In [1]:
import os
import os.path
import sys
import pandas as pd
import numpy as np
from importlib import reload

In [2]:
LOCAL_PATH = 'data'
RAW_TRACE = 'bilibili.pcapng'
TCP_TRACE_FEATURE_FILE = 'bilibili_tcp_pkt.csv'
UDP_TRACE_FEATURE_FILE = 'bilibili_udp_pkt.csv'
BUCKET_NAME = 'edu.nyu.hsn.ddos-data' # replace with your bucket name
KEY = 'CAP_NIC1_00931_20130727230801.dms' # replace with your object key

In [3]:
if not os.path.exists(os.path.join(LOCAL_PATH, RAW_TRACE)):
    if not os.path.exists(LOCAL_PATH):
        os.mkdir(LOCAL_PATH)
    
    import boto3
    import botocore

    s3 = boto3.resource('s3')

    try:
        s3.Bucket(BUCKET_NAME).download_file(KEY, os.path.join(LOCAL_PATH, RAW_TRACE))
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist.")
        else:
            raise

In [15]:
# convert raw trace to readable udp and tcp packet feature csv file
from python import packet_feature
reload(packet_feature)
%time packet_feature.udp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) , os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
%time packet_feature.tcp_generate(os.path.join(LOCAL_PATH, RAW_TRACE) ,os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))

Conversion done
CPU times: user 3.04 ms, sys: 7.24 ms, total: 10.3 ms
Wall time: 8.59 s
Conversion done
CPU times: user 2.23 s, sys: 141 ms, total: 2.37 s
Wall time: 12.6 s


In [17]:
SAMPLING_RATES = [100]

In [ ]:
from python import flow_feature
reload(flow_feature)
def sample_pkt_to_csv(sampling_rates, trace_df, extract_func):
    for sampling_percent in tqdm_notebook(sampling_rates,desc='Sampling'):
        sampling_rate = sampling_percent / 100.0
        sampled_df = sample_trace(trace_df, sampling_rate)
        if sampling_percent < 100:
            extract_func(sampled_df, sampling_rate, upsampled=False).to_csv(os.path.join(LOCAL_PATH,'packet_rand_{PERCENT}%_no_upsampling.csv'.format(PERCENT=sampling_percent)))
            extract_func(sampled_df, sampling_rate, upsampled=True).to_csv(os.path.join(LOCAL_PATH,'packet_rand_{PERCENT}%_with_upsampling.csv'.format(PERCENT=sampling_percent)))
        elif sampling_percent == 100:
            extract_func(sampled_df, sampling_rate, upsampled=False).to_csv('{PATH}/packet_full.csv'.format(PATH=LOCAL_PATH))
        else:
            raise

In [19]:
from python import flow_feature
reload(flow_feature)

# packet-based random sampling
tcp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, TCP_TRACE_FEATURE_FILE))
udp_trace_df = pd.read_csv(os.path.join(LOCAL_PATH, UDP_TRACE_FEATURE_FILE))
for trace_df in [tcp_trace_df, udp_trace_df]:
    if (tcp_trace_df == trace_df):
        sample_pkt_to_csv(100.0, trace_df, tcp_generate)
    elif (udp_trace_df == trace_df):
        sample_pkt_to_csv(100.0, trace_df, udp_generate)
    else:
        raise

HBox(children=(IntProgress(value=0, description='Sampling', max=1), HTML(value='')))

NameError: name 'sample_trace' is not defined